In [1]:
import os
import random

from ultralytics import YOLO
import cv2
import logging
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import zipfile
import numpy as np
from skimage import io
from skimage.restoration import denoise_tv_chambolle
import math

In [2]:


# def unzip_files(zip_file_path):
#     with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#         zip_ref.extractall()


# zip_file_path = 'archive.zip'


# unzip_files(zip_file_path)

In [3]:
def load_labels(image_file, train_labels):
    label_file = os.path.splitext(image_file)[0] + ".txt"
    label_path = os.path.join(train_labels, label_file)
    
    with open(label_path, "r") as f:
        labels = f.read().strip().split("\n")
    
    return labels

def plot_object_detections(ax, image, labels):
    for label in labels:
        if len(label.split()) != 5:
            continue
        class_id, x_center, y_center, width, height = map(float, label.split())
        x_min = int((x_center - width/2) * image.shape[1])
        y_min = int((y_center - height/2) * image.shape[0])
        x_max = int((x_center + width/2) * image.shape[1])
        y_max = int((y_center + height/2) * image.shape[0])
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 3)

    ax.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    ax.axis('off')




In [9]:
# random_images = random.sample(image_files, 16)
model = YOLO("yolov8x.pt") 



results = model.val(data='data_1.yaml')
   

Ultralytics YOLOv8.2.2  Python-3.9.13 torch-2.0.0 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 19.1MB/s]
val: Scanning C:\Users\Sway\ece50024\datasets\valid\labels... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<00:00, 872.91it/s]

val: WARNING  C:\Users\Sway\ece50024\datasets\valid\images\Pias--359-_PNG.rf.43bcf36efe5cf8c37552d2c45fffea60.jpg: 9 duplicate labels removed
val: WARNING  C:\Users\Sway\ece50024\datasets\valid\images\Pias--360-_PNG.rf.8405b0e44009a9300e0a1100ccf7d5b3.jpg: 7 duplicate labels removed
val: New cache created: C:\Users\Sway\ece50024\datasets\valid\labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:09<00:00,  1.96it/s]


                   all        300       2568    0.00709     0.0629    0.00419    0.00327
            motorcycle        300         32    0.00516      0.906    0.00683    0.00293
              airplane        300        425     0.0833    0.00235     0.0463     0.0417
                   bus        300        842     0.0145     0.0451    0.00757    0.00486
                  boat        300          2          0          0          0          0
         traffic light        300        110          0          0          0          0
          fire hydrant        300        335          0          0          0          0
             stop sign        300        142          0          0          0          0
         parking meter        300          1          0          0          0          0
                 bench        300        192     0.0105     0.0521    0.00633    0.00276
                  bird        300          1          0          0          0          0
                   ca

In [4]:

# Functions for image -> image np array

def image_to_normalized_array(image):
    """
    Converts a PIL Image object to a normalized (0.0 to 1.0) float32 image array.

    Parameters:
        image (PIL.Image): The input PIL Image object.

    Returns:
        numpy.ndarray: The normalized image array with pixel values between 0.0 and 1.0.
    """
    # Convert the PIL Image to a NumPy array
    img_array = np.asarray(image)

    # Convert the array's data type to float32 for normalization
    img_array = img_array.astype(np.float32)

    # Normalize pixel values to be between 0.0 and 1.0
    img_array /= 255.0

    return img_array

In [31]:
# Best version

# Plug-and-Play ADMM denoising


def update_x(x_prev, v_prev, u_prev, rho, f):
    x = (f + rho * (v_prev - u_prev)) / (1 + rho)
    return x

def denoiser(image, sigma):
    return denoise_tv_chambolle(image, weight=sigma)

def update_v(x, u_prev, lambda_val, rho):
    sigma = np.sqrt(lambda_val / rho)
    return denoiser(x + u_prev, sigma)

def update_u(u_prev, x, v):
    return u_prev + (x - v)

def calc_delta(x, x_prev, v, v_prev, u, u_prev):
    delta = np.sqrt(np.mean((x - x_prev)**2) + np.mean((v - v_prev)**2) + np.mean((u - u_prev)**2))
    return delta

def plug_and_play_admm(f, lambda_val, rho_0, eta, gamma, tol, max_iterations):
    x = f.copy()
    v = f.copy()
    u = np.zeros_like(f)

    rho = rho_0
    delta = np.inf
    k = 0

    while delta >= tol and k < max_iterations:
        x_prev = x.copy()
        v_prev = v.copy()
        u_prev = u.copy()

        x = update_x(x_prev, v_prev, u_prev, rho, f)
        v = update_v(x, u_prev, lambda_val, rho)
        u = update_u(u_prev, x, v)

        delta_prev = delta
        delta = calc_delta(x, x_prev, v, v_prev, u, u_prev)

        if delta >= eta * delta_prev:
            rho *= gamma

        k += 1

    return x

def noise_gray(image, noise_level):
  noise = np.random.normal(scale=noise_level, size=image.shape)
  noisy_image = image + noise
  noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)
  return noisy_image

# image = cv2.imread(dir_path + 'dog_on_grass.jpg')
# image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# image_array = image_to_normalized_array(image_gray)

# noisy_image = noise_gray(image_gray, 30)
# noisy_array = image_to_normalized_array(noisy_image)

# Example usage:
# f: Blurred image
# lambda_val: Regularization parameter
# rho_0: Initial value of rho
# eta: Convergence criteria parameter
# gamma: Parameter for adjusting rho
# tol: Convergence tolerance
# max_iterations: Maximum number of iterations
# deblurred_image = plug_and_play_admm(f, lambda_val, rho_0, eta, gamma, tol, max_iterations)
# denoised_array = plug_and_play_admm(noisy_array, lambda_val=0.001, rho_0=0.00001, eta=0.5, gamma=1.4, tol=0.001, max_iterations=100)


# # Calculate PSNR and SSIM for noisy image
# psnr_noisy, ssim_noisy = compare_images(img_array, noisy_array)
# psnr_denoised, ssim_denoised = compare_images(img_array, denoised_array)
# print(f"Noisy Image PSNR: {psnr_noisy:.2f}, SSIM: {ssim_noisy:.4f}")
# print(f"Denoised Image PSNR: {psnr_denoised:.2f}, SSIM: {ssim_denoised:.4f}")

# # Display the deblurred image
# plt.figure(figsize=(16, 8))

# plt.subplot(1, 3, 1)
# plt.imshow(image_array, cmap='gray')
# plt.title('Original Image')

# plt.subplot(1, 3, 2)
# plt.imshow(noisy_array, cmap='gray')
# plt.title('Blurred Image')

# plt.subplot(1, 3, 3)
# plt.imshow(denoised_array, cmap='gray')
# plt.title('Deblurred Image')

# plt.show()


SyntaxError: invalid syntax (1925093448.py, line 65)

In [22]:
def add_noise_and_save(dir_path):
    for filename in os.listdir(dir_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img_path = os.path.join(dir_path, filename)
            image = cv2.imread(img_path)
            image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            noisy_image = noise_gray(image_gray, 30)
            noisy_img_path = os.path.join("gray_noise_images", '' + filename)
            cv2.imwrite(noisy_img_path, noisy_image)
add_noise_and_save('original/')

In [23]:
results = model.val(data='data_1.yaml')

Ultralytics YOLOv8.2.2  Python-3.9.13 torch-2.0.0 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)


val: Scanning C:\Users\Sway\ece50024\datasets\valid\labels... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<00:00, 956.02it/s]

val: WARNING  C:\Users\Sway\ece50024\datasets\valid\images\Pias--359-_PNG.rf.43bcf36efe5cf8c37552d2c45fffea60.jpg: 9 duplicate labels removed
val: WARNING  C:\Users\Sway\ece50024\datasets\valid\images\Pias--360-_PNG.rf.8405b0e44009a9300e0a1100ccf7d5b3.jpg: 7 duplicate labels removed
val: New cache created: C:\Users\Sway\ece50024\datasets\valid\labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:09<00:00,  2.10it/s]


                   all        300       2568    0.00937     0.0354    0.00586    0.00283
            motorcycle        300         32     0.0048      0.375    0.00498    0.00215
              airplane        300        425     0.0392    0.00941     0.0202     0.0141
                   bus        300        842     0.0125     0.0428     0.0066    0.00379
                  boat        300          2          0          0          0          0
         traffic light        300        110          0          0          0          0
          fire hydrant        300        335     0.0706     0.0896     0.0427     0.0168
             stop sign        300        142          0          0          0          0
         parking meter        300          1          0          0          0          0
                 bench        300        192      0.014     0.0417     0.0145    0.00606
                  bird        300          1          0          0          0          0
                   ca

In [33]:
import os
import cv2
import numpy as np

def denoise_and_save(input_dir_path, output_dir_path):
    for filename in os.listdir(input_dir_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            
            img_path = os.path.join(input_dir_path, filename)
            image = cv2.imread(img_path)
            
            image_array = image_to_normalized_array(image)

            
            denoised_array = plug_and_play_admm(image_array, lambda_val=0.001, rho_0=0.00001, eta=0.5, gamma=1.4, tol=0.001, max_iterations=100)

           
            denoised_image = (denoised_array * 255).astype(np.uint8)

          
            denoised_img_path = os.path.join(output_dir_path, '' + filename)
            cv2.imwrite(denoised_img_path, denoised_image)


# denoise_and_save('gray_noise_images', 'gray_denoise_images')


In [6]:
model = YOLO("yolov8x.pt") 
results = model.val(data='data_1.yaml')

Ultralytics YOLOv8.2.2  Python-3.9.13 torch-2.0.0 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs


val: Scanning C:\Users\Sway\ece50024\datasets\valid\labels... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<00:00, 519.66it/s]

val: WARNING  C:\Users\Sway\ece50024\datasets\valid\images\Pias--359-_PNG.rf.43bcf36efe5cf8c37552d2c45fffea60.jpg: 9 duplicate labels removed
val: WARNING  C:\Users\Sway\ece50024\datasets\valid\images\Pias--360-_PNG.rf.8405b0e44009a9300e0a1100ccf7d5b3.jpg: 7 duplicate labels removed
val: New cache created: C:\Users\Sway\ece50024\datasets\valid\labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:14<00:00,  1.32it/s]


                   all        300       2568    0.00706     0.0424    0.00448     0.0021
            motorcycle        300         32    0.00435      0.469    0.00458    0.00208
              airplane        300        425          0          0          0          0
                   bus        300        842     0.0274     0.0641     0.0147    0.00857
                  boat        300          2          0          0          0          0
         traffic light        300        110          0          0          0          0
          fire hydrant        300        335     0.0525     0.0627     0.0335     0.0145
             stop sign        300        142          0          0          0          0
         parking meter        300          1          0          0          0          0
                 bench        300        192     0.0116     0.0312    0.00981    0.00307
                  bird        300          1          0          0          0          0
                   ca